# RAGAS Metrics 이해하기
- [필수 사항] 이 노트북을 실행하기 이전에 setup/README.md 를 참고하여 "가상환경" 을 먼저 설치하시고, 이 가상 환경을 커널로 설정 후에 진행 하세요.
- 참고 : RAGAS Metrics
    - [Context Precision](https://docs.ragas.io/en/latest/concepts/metrics/available_metrics/context_precision/)
    - [Context Recall](https://docs.ragas.io/en/latest/concepts/metrics/available_metrics/context_recall/)
    - [Response Relevancy](https://docs.ragas.io/en/latest/concepts/metrics/available_metrics/answer_relevance/)
    - [Faithfulness](https://docs.ragas.io/en/latest/concepts/metrics/available_metrics/faithfulness/)
    - [Factual Correctness](https://docs.ragas.io/en/latest/concepts/metrics/available_metrics/factual_correctness/)
    - [SQL](https://docs.ragas.io/en/latest/concepts/metrics/available_metrics/sql/)

# 1. RAGAS 래핑 모델 생성

In [10]:
import boto3
from datasets import Dataset

from langchain_aws import ChatBedrockConverse
from ragas import evaluate

# Bedrock 클라이언트 설정
bedrock_client = boto3.client(
    service_name='bedrock-runtime',
    region_name='us-west-2'
)

from ragas.llms import LangchainLLMWrapper

evaluator_llm = LangchainLLMWrapper(ChatBedrockConverse(
    model="anthropic.claude-3-5-haiku-20241022-v1:0", 
    client=bedrock_client,
))

from langchain_community.embeddings import BedrockEmbeddings
# from ragas.embeddings import LangchainEmbeddingWrapper
from ragas.embeddings import LangchainEmbeddingsWrapper

# Bedrock Embeddings 설정
evaluator_embeddings = BedrockEmbeddings(
    client=bedrock_client,
    model_id="amazon.titan-embed-text-v1"  # 또는 다른 임베딩 모델
)

# RAGAS Wrapper로 감싸기
embeddings_wrapper = LangchainEmbeddingsWrapper(base_embeddings)

# 2. Context Precision
- Context Precision은 retrieved_contexts에서 관련된 청크들의 비율을 측정하는 지표입
- 문서나 텍스트를 검색할 때, 시스템은 여러 개의 작은 조각(청크)들을 가져옵니다. 이때 우리가 원하는 정보와 관련 있는 조각들을 얼마나 정확하게 가져왔는지를 측정하는 것이 Context Precision입니다.
    - 예를 들어 설명하면: 당신이 "한국의 전통 음식"에 대해 검색했다고 가정해봅시다. 
    - 시스템이 10개의 텍스트 조각을 가져왔는데, 그 중 7개는 실제로 한국 음식에 대한 내용이고,  3개는 다른 나라의 음식이나 관련 없는 내용이라면 이 경우의 Context Precision은 7/10 = 0.7 또는 70%가 됩니다

## Context Precision without reference

In [3]:
from ragas import SingleTurnSample
from ragas.metrics import LLMContextPrecisionWithoutReference

context_precision = LLMContextPrecisionWithoutReference(llm=evaluator_llm)

sample = SingleTurnSample(
    user_input="Where is the Eiffel Tower located?",
    response="The Eiffel Tower is located in Paris.",
    retrieved_contexts=["The Eiffel Tower is located in Paris."], 
)


await context_precision.single_turn_ascore(sample)

0.9999999999

## Context Precision with reference

In [4]:
from ragas import SingleTurnSample
from ragas.metrics import LLMContextPrecisionWithReference

context_precision = LLMContextPrecisionWithReference(llm=evaluator_llm)

sample = SingleTurnSample(
    user_input="Where is the Eiffel Tower located?",
    reference="The Eiffel Tower is located in Paris.",
    retrieved_contexts=["The Eiffel Tower is located in Paris."], 
)

await context_precision.single_turn_ascore(sample)

0.9999999999

## Non LLM Based Context Precision

In [5]:
from ragas import SingleTurnSample
from ragas.metrics import NonLLMContextPrecisionWithReference

context_precision = NonLLMContextPrecisionWithReference()

sample = SingleTurnSample(
    retrieved_contexts=["The Eiffel Tower is located in Paris."], 
    reference_contexts=["Paris is the capital of France.", "The Eiffel Tower is one of the most famous landmarks in Paris."]
)

await context_precision.single_turn_ascore(sample)

0.9999999999

# 3. Context Recall
Context Recall은 관련된 문서(또는 정보)들이 얼마나 성공적으로 검색되었는지를 측정합니다. 이는 중요한 결과들을 놓치지 않는 것에 초점을 맞춥니다. 높은 recall은 관련된 문서들이 더 적게 누락되었다는 것을 의미합니다. 간단히 말해서, recall은 중요한 정보를 놓치지 않는 것에 관한 것입니다. 이는 누락된 것이 없는지를 측정하는 것이기 때문에, context recall을 계산하기 위해서는 항상 비교할 수 있는 기준이 필요합니다.

더 쉽게 설명하면:
- Precision이 "가져온 정보가 얼마나 정확한가"를 측정한다면
- Recall은 "필요한 정보를 얼마나 빠짐없이 가져왔는가"를 측정합니다

예를 들어:
- 도서관에 한국 요리에 대한 책이 총 100권이 있다고 가정했을 때
- 검색 시스템이 80권을 찾아냈다면
- Recall은 80/100 = 0.8 또는 80%가 됩니다
- 즉, 필요한 정보의 80%를 찾아냈다는 의미입니다

## LLM Based Context Recall

In [6]:
from ragas.dataset_schema import SingleTurnSample
from ragas.metrics import LLMContextRecall

sample = SingleTurnSample(
    user_input="Where is the Eiffel Tower located?",
    response="The Eiffel Tower is located in Paris.",
    reference="The Eiffel Tower is located in Paris.",
    retrieved_contexts=["Paris is the capital of France."], 
)

context_recall = LLMContextRecall(llm=evaluator_llm)
await context_recall.single_turn_ascore(sample)

1.0

# 4. Response Relevancy
`ResponseRelevancy` 지표는 응답이 사용자 입력과 얼마나 관련이 있는지를 측정합니다. 높은 점수는 사용자 입력과의 더 나은 일치를 나타내며, 응답이 불완전하거나 불필요한 정보를 포함할 경우 낮은 점수가 주어집니다.

이 지표는 `user_input`과 `response`를 사용하여 다음과 같이 계산됩니다:
1. 응답을 기반으로 인공적인 질문들(기본값 3개)을 생성합니다. 이 질문들은 응답의 내용을 반영하도록 설계됩니다.
2. 사용자 입력의 임베딩(Eo)과 각 생성된 질문의 임베딩(Egi) 사이의 코사인 유사도를 계산합니다.
3. 이러한 코사인 유사도 점수들의 평균을 계산하여 **응답 관련성(Answer Relevancy)**을 구합니다.

더 쉽게 설명하면:
- 이 지표는 "시스템의 응답이 사용자의 질문이나 요청에 얼마나 잘 부합하는가"를 측정합니다
- 예를 들어, 사용자가 "김치 만드는 방법"을 물었는데:
  - 김치 만드는 과정을 자세히 설명하면 높은 점수
  - 다른 한식 요리법을 설명하거나 불필요한 정보를 포함하면 낮은 점수를 받게 됩니다

In [12]:
from ragas import SingleTurnSample 
from ragas.metrics import ResponseRelevancy

sample = SingleTurnSample(
        user_input="When was the first super bowl?",
        response="The first superbowl was held on Jan 15, 1967",
        retrieved_contexts=[
            "The First AFL–NFL World Championship Game was an American football game played on January 15, 1967, at the Los Angeles Memorial Coliseum in Los Angeles."
        ]
    )

scorer = ResponseRelevancy(llm=evaluator_llm, embeddings=evaluator_embeddings)
await scorer.single_turn_ascore(sample)

0.9486586568274785

# 5. Faithfulness
**Faithfulness**(충실도) 지표는 `응답`이 `검색된 컨텍스트`와 얼마나 사실적으로 일치하는지를 측정합니다. 0에서 1 사이의 값을 가지며, 높은 점수일수록 더 나은 일관성을 나타냅니다.

응답이 **충실하다(faithful)**고 판단되는 것은 응답의 모든 주장이 검색된 컨텍스트로부터 뒷받침될 수 있을 때입니다.

계산 방법:
1. 응답에 있는 모든 주장들을 식별합니다
2. 각 주장이 검색된 컨텍스트로부터 추론될 수 있는지 확인합니다
3. 공식을 사용하여 충실도 점수를 계산합니다

더 쉽게 설명하면:
- 이는 "LLM이 제공한 응답이 Retrieved Context의 내용과 얼마나 일치하는가"를 측정합니다
- 예를 들어:
  - Retrieved Context에 "김치는 배추, 고춧가루, 마늘이 들어갑니다"라고 되어있는데
  - LLM 이 "김치의 주재료는 배추, 고춧가루, 마늘입니다"라고 응답하면 높은 충실도 점수
  - 반면 "김치에는 당근이 들어갑니다"라고 응답하면 낮은 충실도 점수를 받게 됩니다

In [13]:
from ragas.dataset_schema import SingleTurnSample 
from ragas.metrics import Faithfulness

sample = SingleTurnSample(
        user_input="When was the first super bowl?",
        response="The first superbowl was held on Jan 15, 1967",
        retrieved_contexts=[
            "The First AFL–NFL World Championship Game was an American football game played on January 15, 1967, at the Los Angeles Memorial Coliseum in Los Angeles."
        ]
    )
scorer = Faithfulness(llm=evaluator_llm)
await scorer.single_turn_ascore(sample)

1.0

# 6. Factual Correctness
`FactualCorrectness`(사실적 정확도) 지표는 생성된 `응답`의 사실적 정확성을 `참조`와 비교하고 평가합니다. 이 지표는 생성된 응답이 참조와 얼마나 일치하는지를 판단하는 데 사용됩니다. 사실적 정확도 점수는 0에서 1 사이의 값을 가지며, 높은 값일수록 더 나은 성능을 나타냅니다.

응답과 참조 간의 일치도를 측정하기 위해, 이 지표는 다음과 같은 과정을 거칩니다:
1. LLM(대규모 언어 모델)을 사용하여 응답과 참조를 각각의 주장들로 분해합니다
2. 자연어 추론을 사용하여 응답과 참조 사이의 사실적 중복을 판단합니다
3. 사실적 중복은 정밀도(precision), 재현율(recall), F1 점수를 사용하여 수치화됩니다

더 쉽게 설명하면:
- 이 지표는 "LLM의 응답이 정답과 얼마나 사실적으로 일치하는가"를 측정합니다
- 예를 들어:
  - 정답(참조)이 "서울은 대한민국의 수도이며 약 1000만 명의 인구가 있습니다"일 때
  - LLM 응답이 "서울은 대한민국의 수도이고 인구가 약 1000만 명입니다"라면 높은 점수
  - 반면 "서울은 대한민국의 수도이고 인구가 500만 명입니다"라면 낮은 점수를 받게 됩니다

In [14]:
from ragas.dataset_schema import SingleTurnSample
from ragas.metrics._factual_correctness import FactualCorrectness


sample = SingleTurnSample(
    response="The Eiffel Tower is located in Paris.",
    reference="The Eiffel Tower is located in Paris. I has a height of 1000ft."
)

scorer = FactualCorrectness(llm = evaluator_llm)
await scorer.single_turn_ascore(sample)

0.5

# 7. SQL
## 7.1. Execution based metrics
이러한 지표들에서는 SQL 쿼리를 데이터베이스에서 실행한 후 그 `응답`을 예상 결과와 비교하여 평가합니다.

DataCompy 점수:
`DataCompyScore` 지표는 두 개의 pandas DataFrame을 비교하는 파이썬 라이브러리인 DataCompy를 사용합니다. 이는 두 DataFrame을 비교하고 차이점에 대한 상세한 보고서를 제공하는 간단한 인터페이스를 제공합니다. 이 지표에서는 `응답`을 데이터베이스에서 실행하고 그 결과 데이터를 예상 데이터(`reference`)와 비교합니다. 비교를 가능하게 하기 위해 `응답`과 `reference` 모두 예시와 같이 쉼표로 구분된 값(CSV) 형태여야 합니다.

DataFrame은 행 또는 열 단위로 비교할 수 있습니다. 이는 `mode` 매개변수를 사용하여 설정할 수 있습니다:
- `mode`가 `row`이면 행 단위로 비교가 수행됩니다
- `mode`가 `column`이면 열 단위로 비교가 수행됩니다

더 쉽게 설명하면:
- 이는 "SQL 쿼리 실행 결과가 예상했던 결과와 얼마나 일치하는지"를 측정합니다
- 예를 들어:
  - 학생 성적 데이터베이스에서 "A학점 받은 학생 목록"을 조회하는 경우
  - 정답 쿼리의 결과와 시스템이 생성한 쿼리의 결과를 비교하여
  - 동일한 학생들이 나오는지, 정보가 정확한지 등을 검증합니다
  - 이때 행 단위로 비교하면 각 학생의 전체 정보가 일치하는지를
  - 열 단위로 비교하면 특정 항목(예: 이름, 학점 등)별로 일치하는지를 확인할 수 있습니다

In [17]:
from ragas.metrics import DataCompyScore
from ragas.dataset_schema import SingleTurnSample

data1 = """acct_id,dollar_amt,name,float_fld,date_fld
10000001234,123.45,George Maharis,14530.1555,2017-01-01
10000001235,0.45,Michael Bluth,1,2017-01-01
10000001236,1345,George Bluth,,2017-01-01
10000001237,123456,Bob Loblaw,345.12,2017-01-01
10000001238,1.05,Lucille Bluth,,2017-01-01
10000001238,1.05,Loose Seal Bluth,,2017-01-01
"""

data2 = """acct_id,dollar_amt,name,float_fld
10000001234,123.4,George Michael Bluth,14530.155
10000001235,0.45,Michael Bluth,
10000001236,1345,George Bluth,1
10000001237,123456,Robert Loblaw,345.12
10000001238,1.05,Loose Seal Bluth,111
"""
sample = SingleTurnSample(response=data1, reference=data2)
scorer = DataCompyScore()
await scorer.single_turn_ascore(sample)

INFO|2025-01-31 03:03:45,176|datacompy.core|core.py|_compare|212: df1 does not Pandas.DataFrame.equals df2
INFO|2025-01-31 03:03:45,176|datacompy.core|core.py|_compare|213: Number of columns in common: 4
INFO|2025-01-31 03:03:45,177|datacompy.core|core.py|_compare|217: Number of columns in df1 and not in df2: 0
INFO|2025-01-31 03:03:45,177|datacompy.core|core.py|_compare|221: Column in df2 and not in df1: date_fld
INFO|2025-01-31 03:03:45,178|datacompy.core|core.py|_compare|222: Number of columns in df2 and not in df1: 1
INFO|2025-01-31 03:03:45,184|datacompy.core|core.py|_dataframe_merge|326: Number of rows in df1 and not in df2: 0
INFO|2025-01-31 03:03:45,185|datacompy.core|core.py|_dataframe_merge|327: Number of rows in df2 and not in df1: 1
INFO|2025-01-31 03:03:45,186|datacompy.core|core.py|_dataframe_merge|331: Number of rows in df1 and df2 (not necessarily equal): 5
INFO|2025-01-31 03:03:45,188|datacompy.core|core.py|_intersect_compare|375: acct_id: 5 / 5 (100.00%) match
INFO|20

nan

## 7.2 Non Execution based metrics
SQL 쿼리를 데이터베이스에서 실행하는 것은 시간이 많이 걸리고 때로는 실행이 어려울 수 있습니다. 이러한 경우에는 실행하지 않고 평가하는 지표들을 사용할 수 있습니다. 이러한 지표들은 SQL 쿼리를 데이터베이스에서 실행하지 않고 직접 비교합니다.

SQL 쿼리 의미적 동등성:
`LLMSQLEquivalence`는 `응답` 쿼리와 `참조` 쿼리의 동등성을 평가하는데 사용되는 지표입니다. 이 지표는 쿼리를 비교할 때 사용할 데이터베이스 스키마도 필요하며, 이는 `reference_contexts`에 입력됩니다. 이는 이진 지표로, 1은 SQL 쿼리들이 의미적으로 동등함을, 0은 SQL 쿼리들이 의미적으로 동등하지 않음을 나타냅니다.

더 쉽게 설명하면:
- 이는 "두 SQL 쿼리가 실제로 실행하지 않고도 같은 결과를 만들어낼 것인지"를 평가합니다
- 예를 들어:
  - "학생들의 평균 성적을 구하는" 두 개의 다른 SQL 쿼리가 있을 때
  - 쿼리의 작성 방식은 다르더라도
  - 같은 결과를 얻을 수 있다면 의미적으로 동등(점수 1)
  - 다른 결과가 나온다면 동등하지 않음(점수 0)으로 판단합니다
- 이는 실제로 쿼리를 실행하지 않고도 빠르게 평가할 수 있는 장점이 있습니다

In [19]:
from ragas.metrics import LLMSQLEquivalence
from ragas.dataset_schema import SingleTurnSample

sample = SingleTurnSample(
    response="""
        SELECT p.product_name, SUM(oi.quantity) AS total_quantity
        FROM order_items oi
        JOIN products p ON oi.product_id = p.product_id
        GROUP BY p.product_name;
    """,
    reference="""
        SELECT p.product_name, COUNT(oi.quantity) AS total_quantity
        FROM order_items oi
        JOIN products p ON oi.product_id = p.product_id
        GROUP BY p.product_name;
    """,
    reference_contexts=[
        """
        Table order_items:
        - order_item_id: INT
        - order_id: INT
        - product_id: INT
        - quantity: INT
        """,
        """
        Table products:
        - product_id: INT
        - product_name: VARCHAR
        - price: DECIMAL
        """
    ]
)

scorer = LLMSQLEquivalence()
scorer.llm = evaluator_llm
await scorer.single_turn_ascore(sample)

0